# Anaconda Server Diagnostics

This notebook helps diagnose issues with the Plotbot installation scripts, specifically problems connecting to Anaconda servers.

## Issue Description
The `1_init_conda.sh` script hangs when trying to connect to the anaconda server, and the server "looks different" according to user reports.

## Troubleshooting Steps

Let's run through a series of diagnostic checks to understand what's happening.


## 1. Basic Connectivity Tests

In [1]:
# Test basic internet connectivity
import subprocess
import socket
import sys
import requests
from datetime import datetime

print(f"Diagnostic run at: {datetime.now()}")
print(f"Python version: {sys.version}")
print("="*50)

# Test basic internet connectivity
def test_connectivity(host, port=80, timeout=5):
    try:
        socket.create_connection((host, port), timeout)
        return True
    except OSError:
        return False

print("\n🔗 Testing basic connectivity:")
hosts_to_test = [
    ("google.com", 80),
    ("anaconda.org", 80),
    ("anaconda.org", 443),
    ("conda.anaconda.org", 80),
    ("conda.anaconda.org", 443),
    ("repo.anaconda.com", 80),
    ("repo.anaconda.com", 443)
]

for host, port in hosts_to_test:
    status = "✅" if test_connectivity(host, port) else "❌"
    print(f"{status} {host}:{port}")


Diagnostic run at: 2025-06-06 09:32:50.800592
Python version: 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]

🔗 Testing basic connectivity:
✅ google.com:80
✅ anaconda.org:80
✅ anaconda.org:443
✅ conda.anaconda.org:80
✅ conda.anaconda.org:443
✅ repo.anaconda.com:80
✅ repo.anaconda.com:443


## 2. Ping Tests (Like the Script Does)

In [2]:
# Test ping commands that the script uses
import subprocess
import time

def run_ping_test(host, count=1, timeout=10):
    print(f"\n🏓 Ping test for {host}:")
    try:
        start_time = time.time()
        result = subprocess.run(
            ["ping", "-c", str(count), host], 
            capture_output=True, 
            text=True, 
            timeout=timeout
        )
        end_time = time.time()
        
        print(f"   Time taken: {end_time - start_time:.2f} seconds")
        print(f"   Return code: {result.returncode}")
        
        if result.returncode == 0:
            print(f"   ✅ Success")
            print(f"   Output: {result.stdout.strip().split(chr(10))[-2]}")
        else:
            print(f"   ❌ Failed")
            if result.stderr:
                print(f"   Error: {result.stderr.strip()}")
        
        return result.returncode == 0
        
    except subprocess.TimeoutExpired:
        print(f"   ⏰ Timeout after {timeout} seconds - THIS IS LIKELY THE ISSUE!")
        return False
    except Exception as e:
        print(f"   💥 Exception: {e}")
        return False

# Test the exact same command the script uses
ping_success = run_ping_test("anaconda.org")

# Test other related hosts
run_ping_test("conda.anaconda.org")
run_ping_test("repo.anaconda.com")

print(f"\n📊 Summary: anaconda.org ping {'succeeded' if ping_success else 'failed'}")



🏓 Ping test for anaconda.org:
   Time taken: 0.05 seconds
   Return code: 0
   ✅ Success
   Output: 1 packets transmitted, 1 packets received, 0.0% packet loss

🏓 Ping test for conda.anaconda.org:
   Time taken: 0.05 seconds
   Return code: 0
   ✅ Success
   Output: 1 packets transmitted, 1 packets received, 0.0% packet loss

🏓 Ping test for repo.anaconda.com:
   Time taken: 0.05 seconds
   Return code: 0
   ✅ Success
   Output: 1 packets transmitted, 1 packets received, 0.0% packet loss

📊 Summary: anaconda.org ping succeeded


## 3. HTTP/HTTPS Response Tests

In [3]:
# Test HTTP responses from anaconda servers
import requests
import time

def test_http_response(url, timeout=10):
    print(f"\n🌐 Testing HTTP response for {url}:")
    try:
        start_time = time.time()
        response = requests.get(url, timeout=timeout)
        end_time = time.time()
        
        print(f"   Time taken: {end_time - start_time:.2f} seconds")
        print(f"   Status code: {response.status_code}")
        print(f"   Server: {response.headers.get('Server', 'Unknown')}")
        print(f"   Content-Type: {response.headers.get('Content-Type', 'Unknown')}")
        
        if response.status_code == 200:
            print(f"   ✅ Success")
            # Show first 200 characters of response
            preview = response.text[:200].replace('\n', ' ').strip()
            print(f"   Preview: {preview}...")
        else:
            print(f"   ⚠️ Non-200 status code")
            
        return True
        
    except requests.exceptions.Timeout:
        print(f"   ⏰ Timeout after {timeout} seconds")
        return False
    except requests.exceptions.ConnectionError as e:
        print(f"   🔌 Connection error: {e}")
        return False
    except Exception as e:
        print(f"   💥 Exception: {e}")
        return False

# Test various anaconda URLs
urls_to_test = [
    "https://anaconda.org",
    "http://anaconda.org",
    "https://conda.anaconda.org",
    "https://repo.anaconda.com",
    "https://repo.continuum.io"  # Old URL that might still be referenced
]

for url in urls_to_test:
    test_http_response(url)



🌐 Testing HTTP response for https://anaconda.org:
   Time taken: 0.17 seconds
   Status code: 403
   Server: cloudflare
   Content-Type: text/html; charset=UTF-8
   ⚠️ Non-200 status code

🌐 Testing HTTP response for http://anaconda.org:
   Time taken: 0.26 seconds
   Status code: 403
   Server: cloudflare
   Content-Type: text/html; charset=UTF-8
   ⚠️ Non-200 status code

🌐 Testing HTTP response for https://conda.anaconda.org:
   Time taken: 0.44 seconds
   Status code: 403
   Server: cloudflare
   Content-Type: text/html; charset=UTF-8
   ⚠️ Non-200 status code

🌐 Testing HTTP response for https://repo.anaconda.com:
   Time taken: 0.16 seconds
   Status code: 200
   Server: cloudflare
   Content-Type: text/html
   ✅ Success
   Preview: <!DOCTYPE html> <html>  <head>     <!-- Global site tag (gtag.js) - Google Analytics -->     <script async src="https://www.googletagmanager.com/gtag/js?id=UA-27761864-19"></script>     <script>...

🌐 Testing HTTP response for https://repo.continuum.

## 4. Conda Configuration Analysis

In [4]:
# Check conda configuration and channels
import subprocess
import json

def run_conda_command(cmd, description):
    print(f"\n🐍 {description}:")
    try:
        result = subprocess.run(
            cmd.split(), 
            capture_output=True, 
            text=True, 
            timeout=30
        )
        
        if result.returncode == 0:
            print(f"   ✅ Success")
            output_lines = result.stdout.strip().split('\n')
            for line in output_lines[:10]:  # Show first 10 lines
                print(f"   {line}")
            if len(output_lines) > 10:
                print(f"   ... and {len(output_lines) - 10} more lines")
        else:
            print(f"   ❌ Failed (return code: {result.returncode})")
            if result.stderr:
                print(f"   Error: {result.stderr.strip()}")
                
    except subprocess.TimeoutExpired:
        print(f"   ⏰ Command timed out after 30 seconds")
    except FileNotFoundError:
        print(f"   ❌ Conda not found in PATH")
    except Exception as e:
        print(f"   💥 Exception: {e}")

# Check basic conda info
run_conda_command("conda --version", "Conda version")
run_conda_command("conda info", "Conda info")
run_conda_command("conda config --show channels", "Conda channels")
run_conda_command("conda config --show-sources", "Conda configuration sources")



🐍 Conda version:
   ✅ Success
   conda 24.11.3

🐍 Conda info:
   ✅ Success
   active environment : plotbot_env
       active env location : /opt/anaconda3/envs/plotbot_env
               shell level : 1
          user config file : /Users/robertalexander/.condarc
    populated config files : /Users/robertalexander/.condarc
             conda version : 24.11.3
       conda-build version : 24.5.1
            python version : 3.12.4.final.0
                    solver : libmamba (default)
          virtual packages : __archspec=1=m1
   ... and 19 more lines

🐍 Conda channels:
   ✅ Success
   channels:
     - defaults

🐍 Conda configuration sources:
   ✅ Success
   ==> /Users/robertalexander/.condarc <==
   auto_activate_base: False
   channels:
     - defaults
   
   ==> envvars <==
   allow_softlinks: False


## 5. Test the Problematic Command

In [5]:
# Test the exact command that's causing issues in the script
import subprocess
import time

def test_conda_search(timeout=30):
    print(f"\n🔍 Testing the problematic conda search command:")
    print(f"   Command: conda search conda | grep -E '^conda\\s' | tail -n 1")
    print(f"   Timeout: {timeout} seconds")
    
    try:
        start_time = time.time()
        
        # Run the exact command from the script
        process = subprocess.Popen(
            "conda search conda | grep -E '^conda\\s' | tail -n 1",
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        try:
            stdout, stderr = process.communicate(timeout=timeout)
            end_time = time.time()
            
            print(f"   Time taken: {end_time - start_time:.2f} seconds")
            print(f"   Return code: {process.returncode}")
            
            if process.returncode == 0:
                print(f"   ✅ Success")
                print(f"   Output: '{stdout.strip()}'")
            else:
                print(f"   ❌ Failed")
                if stderr:
                    print(f"   Error: {stderr.strip()}")
                    
        except subprocess.TimeoutExpired:
            print(f"   ⏰ TIMEOUT after {timeout} seconds - THIS IS THE ISSUE!")
            print(f"   The conda search command is hanging, likely due to:")
            print(f"   • Network connectivity issues to anaconda servers")
            print(f"   • DNS resolution problems")
            print(f"   • Firewall/proxy blocking the connection")
            print(f"   • Anaconda servers being slow or unresponsive")
            process.kill()
            
    except Exception as e:
        print(f"   💥 Exception: {e}")

# Test with a shorter timeout first
test_conda_search(timeout=15)

print("\n" + "="*50)
print("RECOMMENDATION: If the conda search command timed out,")
print("modify the script to skip the version check or use offline mode.")



🔍 Testing the problematic conda search command:
   Command: conda search conda | grep -E '^conda\s' | tail -n 1
   Timeout: 15 seconds
   Time taken: 2.38 seconds
   Return code: 0
   ✅ Success
   Output: 'conda                         25.5.1  py39hca03da5_0  pkgs/main'

RECOMMENDATION: If the conda search command timed out,
modify the script to skip the version check or use offline mode.


## 8. System Information

In [8]:
# Gather system information that might be relevant
import platform
import os
import subprocess

print("💻 SYSTEM INFORMATION:")
print(f"Platform: {platform.platform()}")
print(f"System: {platform.system()}")
print(f"Release: {platform.release()}")
print(f"Architecture: {platform.architecture()}")
print(f"Python version: {platform.python_version()}")

# Check environment variables that might affect conda
print("\n🌍 RELEVANT ENVIRONMENT VARIABLES:")
env_vars_to_check = [
    'CONDA_PREFIX', 'CONDA_DEFAULT_ENV', 'CONDA_EXE', 
    'HTTP_PROXY', 'HTTPS_PROXY', 'NO_PROXY',
    'PATH'
]

for var in env_vars_to_check:
    value = os.environ.get(var, 'Not set')
    if var == 'PATH':
        # Show only conda-related PATH entries
        conda_paths = [p for p in value.split(':') if 'conda' in p.lower()]
        print(f"{var} (conda entries): {conda_paths}")
    else:
        print(f"{var}: {value}")

# Check if conda is in PATH
print("\n🔍 CONDA EXECUTABLE LOCATION:")
try:
    conda_path = subprocess.check_output(['which', 'conda'], text=True).strip()
    print(f"conda found at: {conda_path}")
except subprocess.CalledProcessError:
    print("conda not found in PATH")
except FileNotFoundError:
    print("'which' command not available")

print("\n" + "="*50)
print("Sharing this diagnostic information may be helpful")
print("to troubleshoot your installation issues, if it persists!")


💻 SYSTEM INFORMATION:
Platform: macOS-15.5-arm64-arm-64bit
System: Darwin
Release: 24.5.0
Architecture: ('64bit', '')
Python version: 3.12.4

🌍 RELEVANT ENVIRONMENT VARIABLES:
CONDA_PREFIX: /opt/anaconda3/envs/plotbot_env
CONDA_DEFAULT_ENV: plotbot_env
CONDA_EXE: /opt/anaconda3/bin/conda
HTTP_PROXY: Not set
HTTPS_PROXY: Not set
NO_PROXY: Not set
PATH (conda entries): ['/opt/anaconda3/envs/plotbot_env/bin', '/opt/anaconda3/condabin']

🔍 CONDA EXECUTABLE LOCATION:
conda found at: /opt/anaconda3/condabin/conda

Sharing this diagnostic information may be helpful
to troubleshoot your installation issues, if it persists!
